# Benchmark of Speaker Verification Models

In [1]:
from sklearn.metrics import roc_curve
import numpy as np
import os

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format

## Parameters

In [2]:
nets = ['xvector/v001', 'vggvox/v004', 'resnet34vox/v001', 'resnet50vox/v001']

## Benchmark on Vox1-Test

In [3]:
def find_thr_far(far, value):
    return np.argmin(np.abs(value - far))

In [4]:
def format_val(x):
    return str('{:2f}'.format(x))

In [5]:
def count_far(targets, similarities, thr):
    fars = 0
    count = 0
    for t, s in zip(targets, similarities):
        if t == 0:
            if s >= thr:
                fars += 1
            count += 1
    return fars / count

In [6]:
def count_frr(targets, similarities, thr):
    frrs = 0
    count = 0
    for t, s in zip(targets, similarities):
        if t == 1:
            if s < thr:
                frrs += 1
            count += 1
    return frrs / count

In [7]:
vox1_test_results = {}
for net in nets:
    vox1_test_results[net] = pd.read_csv(os.path.join('../data/pt_models', net, 'test_vox1_sv_test.csv'))

In [8]:
results = {}
for net in nets:
    far, tpr, thresholds = roc_curve(vox1_test_results[net]['target'].values, vox1_test_results[net]['similarity'].values)
    frr = 1 - tpr
    idx_eer = np.argmin(np.abs(far - frr))
    idx_far1 = find_thr_far(far, 0.01)
    print(net, thresholds[idx_eer])
    results[net] = [format_val(np.mean([far[idx_eer], frr[idx_eer]])), format_val(far[idx_eer]), format_val(frr[idx_eer]), format_val(thresholds[idx_eer]), 
                    format_val(far[idx_far1]), format_val(frr[idx_far1]), format_val(thresholds[idx_far1]), len(vox1_test_results[net].index)]

xvector/v001 0.8433434963226318
vggvox/v004 0.999350905418396
resnet34vox/v001 0.8419269919395447
resnet50vox/v001 0.9969538450241088


In [14]:
df = pd.DataFrame.from_dict(results, orient='index', columns=['eer', 'eer-far', 'eer-frr', 'eer-thr', 'far1-far', 'far1-frr', 'far1-thr', 'no-trials'])
columns = [('EER','eer'), ('EER','eer-far'), ('EER','eer-frr'), ('EER','eer-thr'), ('FAR1%','far1-far'), ('FAR1%','far1-frr'), ('FAR1%','far1-thr'), ('', 'no-trials')]
df.columns = pd.MultiIndex.from_tuples(columns)
df.style.set_properties(**{'width':'10em', 'text-align':'center'}).set_table_styles([dict(selector="th", props=[('text-align', 'center')])])

## Benchmark on Vox2-MasterVoice-Train

#### Individual Benchmark

In [92]:
setups = [(1, 'raw'), (10, 'any'), (10, 'avg')]

In [93]:
vox2_mv_train_results = {}
for template_dim, policy_name in setups:
    vox2_mv_train_results[policy_name] = {}
    for net in nets:
        vox2_mv_train_results[policy_name][net] = pd.read_csv(os.path.join('../data/pt_models', net, 'test_vox2_mv_train_' + policy_name + '_' + str(template_dim) + '.csv'))
        if policy_name == 'avg':
            vox2_mv_train_results[policy_name][net]['score'] = vox2_mv_train_results[policy_name][net]['score'].apply(lambda x: float(x.split('(')[1].split(',')[0]))
        else:
            vox2_mv_train_results[policy_name][net]['score'] = vox2_mv_train_results[policy_name][net]['score'].apply(lambda x: float(x))

In [94]:
results = {}
for template_dim, policy_name in setups:
    results[policy_name] = {}
    for net in nets:
        far, tpr, thresholds = roc_curve(vox2_mv_train_results[policy_name][net]['target'].values, vox2_mv_train_results[policy_name][net]['score'].values)
        frr = 1 - tpr
        idx_eer = np.argmin(np.abs(far - frr))
        idx_far1 = find_thr_far(far, 0.01)
        results[policy_name][net] = [policy_name] + [net] + [format_val(np.mean([far[idx_eer], frr[idx_eer]])), format_val(far[idx_eer]), format_val(frr[idx_eer]), format_val(thresholds[idx_eer]), 
                                     format_val(far[idx_far1]), format_val(frr[idx_far1]), format_val(thresholds[idx_far1]), len(vox1_test_results[net].index)]

In [95]:
dfs = []
for template_dim, policy_name in setups:
    df = pd.DataFrame.from_dict(results[policy_name], orient='index', columns=['policy', 'net', 'eer', 'eer-far', 'eer-frr', 'eer-thr', 'far1-far', 'far1-frr', 'far1-thr', 'no-trials']) 
    columns = [('-', 'policy'), ('-', 'net'), ('EER','eer'), ('EER','eer-far'), ('EER','eer-frr'), ('EER','eer-thr'), ('FAR1%','far1-far'), ('FAR1%','far1-frr'), ('FAR1%','far1-thr'), ('', 'no-trials')]
    df.columns = pd.MultiIndex.from_tuples(columns)
    dfs.append(df)

In [96]:
df = pd.concat(dfs).reset_index()
del df['index']
df.style.set_properties(**{'width':'10em', 'text-align':'center'}).set_table_styles([dict(selector="th", props=[('text-align', 'center')])])

#### Benchmark based on Raw Policy thresholds

In [77]:
setups = [(1, 'raw'), (10, 'any'), (10, 'avg')]

In [78]:
vox2_mv_train_results = {}
for template_dim, policy_name in setups:
    vox2_mv_train_results[policy_name] = {}
    for net in nets:
        vox2_mv_train_results[policy_name][net] = pd.read_csv(os.path.join('../data/pt_models', net, 'test_vox2_mv_train_' + policy_name + '_' + str(template_dim) + '.csv'))
        if policy_name == 'avg':
            vox2_mv_train_results[policy_name][net]['score'] = vox2_mv_train_results[policy_name][net]['score'].apply(lambda x: float(x.split('(')[1].split(',')[0]))
        else:
            vox2_mv_train_results[policy_name][net]['score'] = vox2_mv_train_results[policy_name][net]['score'].apply(lambda x: float(x))

In [79]:
ref_thrs = {}
for net in nets:
    far, tpr, thresholds = roc_curve(vox2_mv_train_results['raw'][net]['target'].values, vox2_mv_train_results['raw'][net]['score'].values)
    frr = 1 - tpr
    idx_eer = np.argmin(np.abs(far - frr))
    idx_far1 = find_thr_far(far, 0.01)
    ref_thrs[net] = {'eer': thresholds[idx_eer], 'far1': thresholds[idx_far1]}

In [80]:
results = {}
for template_dim, policy_name in setups:
    results[policy_name] = {}
    for net in nets:
        far_eer = count_far(vox2_mv_train_results[policy_name][net]['target'].values, vox2_mv_train_results[policy_name][net]['score'].values, ref_thrs[net]['eer'])
        frr_eer = count_frr(vox2_mv_train_results[policy_name][net]['target'].values, vox2_mv_train_results[policy_name][net]['score'].values, ref_thrs[net]['eer'])
        far_far1 = count_far(vox2_mv_train_results[policy_name][net]['target'].values, vox2_mv_train_results[policy_name][net]['score'].values, ref_thrs[net]['far1'])
        frr_far1 = count_frr(vox2_mv_train_results[policy_name][net]['target'].values, vox2_mv_train_results[policy_name][net]['score'].values, ref_thrs[net]['far1'])
        results[policy_name][net] = [policy_name] + [net] + [format_val(far_eer), format_val(frr_eer), format_val(ref_thrs[net]['eer']), 
                                     format_val(far_far1), format_val(frr_far1), format_val(ref_thrs[net]['far1']), len(vox1_test_results[net].index)]

In [81]:
dfs = []
for template_dim, policy_name in setups:
    df = pd.DataFrame.from_dict(results[policy_name], orient='index', columns=['policy', 'net', 'eer-far', 'eer-frr', 'eer-thr', 'far1-far', 'far1-frr', 'far1-thr', 'no-trials']) 
    columns = [('-', 'policy'), ('-', 'net'), ('EER','eer-far'), ('EER','eer-frr'), ('EER','eer-thr'), ('FAR1%','far1-far'), ('FAR1%','far1-frr'), ('FAR1%','far1-thr'), ('', 'no-trials')]
    df.columns = pd.MultiIndex.from_tuples(columns)
    dfs.append(df)

In [82]:
df = pd.concat(dfs).reset_index()
del df['index']
df.style.set_properties(**{'width':'10em', 'text-align':'center'}).set_table_styles([dict(selector="th", props=[('text-align', 'center')])])

#### Benchmark on Gender based on Raw Policy Thresholds

In [89]:
results = {}
for template_dim, policy_name in setups:
    results[policy_name] = {}
    for net in nets:
        results[policy_name][net] = []
        for gender in ['male', 'female']:
            gender_result = vox2_mv_train_results[policy_name][net][vox2_mv_train_results[policy_name][net]['gender']==gender]
            far_eer = count_far(gender_result['target'].values, gender_result['score'].values, ref_thrs[net]['eer'])
            frr_eer = count_frr(gender_result['target'].values, gender_result['score'].values, ref_thrs[net]['eer'])
            far_far1 = count_far(gender_result['target'].values, gender_result['score'].values, ref_thrs[net]['far1'])
            frr_far1 = count_frr(gender_result['target'].values, gender_result['score'].values, ref_thrs[net]['far1'])
            results[policy_name][net] += [format_val(far_eer), format_val(frr_eer), format_val(far_far1), format_val(frr_far1)]
        results[policy_name][net] = [policy_name] + [net] + results[policy_name][net]

In [90]:
dfs = []
for template_dim, policy_name in setups:
    df = pd.DataFrame.from_dict(results[policy_name], orient='index', columns=['policy', 'net', 'm-eer-far', 'm-eer-frr', 'm-far1-far', 'm-far1-frr', 'f-eer-far', 'f-eer-frr', 'f-far1-far', 'f-far1-frr']) 
    columns = [('-', 'policy'), ('-', 'net'), ('MALE','m-eer-far'), ('MALE','m-eer-frr'), ('MALE','m-far1-far'), ('MALE','m-far1-frr'), 
               ('FEMALE','f-eer-far'), ('FEMALE','f-eer-frr'), ('FEMALE','f-far1-far'), ('FEMALE','f-far1-frr')]
    df.columns = pd.MultiIndex.from_tuples(columns)
    dfs.append(df)

In [91]:
df = pd.concat(dfs).reset_index()
del df['index']
df.style.set_properties(**{'width':'10em', 'text-align':'center'}).set_table_styles([dict(selector="th", props=[('text-align', 'center')])])